In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import concurrent.futures

In [64]:
url = "https://www.macys.com/shop/womens-clothing/accessories?id=29440"

In [92]:
product_brands = []
product_names = []
current_prices = []
old_prices = []
ratings = []
product_urls = []

def scrape_macy(url):

    ua = UserAgent()
    userAgent = ua.random
    headers = {"User-Agent": userAgent}
    page = requests.get(url, headers = headers)
    
    soup = BeautifulSoup(page.content, "html.parser")
    sales_box = soup.find_all("div", class_ = "product-thumbnail-container")

    for sales in sales_box:
        #product_brands
        if sales.find("div", class_ = "product-brand medium") is not None:
            product_brand = sales.find("div", class_ = "product-brand medium")
            product_brands.append(product_brand.text)
        else:
            product_brands.append("None")

        #product_names
        if sales.find("div", class_ = "product-name medium") is not None:
            product_name = sales.find("div", class_ = "product-name medium")
            product_names.append(product_name.text)
        else:
            product_names.append("None")

        #current_prices
        if sales.find("span", attrs ={'v-71178b05': ''} ) is not None:
            current_price = sales.find("span", attrs ={'v-71178b05': ''} )
            current_prices.append(current_price.text)

        elif sales.find("span", class_ = "price-reg is-tier1" ) is not None:
            current_price = sales.find("span", class_ = "price-reg is-tier1" )
            current_prices.append(current_price.text.strip())
        else:
            current_prices.append("None")

        #old_prices
        if sales.find("span", class_ = "price-sm") is not None:
            old_price = sales.find("span", class_ = "price-sm")
            old_prices.append(old_price.text.strip())
        else:
            old_prices.append("None")

        #ratings
    #     if sales.find("div", class_ = "reviewlet-spacing ps-margin-A") is not None:
    #         rating = sales.find("div", class_ = "reviewlet-spacing ps-margin-A")
    #         ratings.append(rating.text)
    #     else:
    #         ratings.append("None")

        if sales.select_one("span.review-star-wrapper") is not None:
            rating = sales.select_one("span.review-star-wrapper")
            rating = rating["aria-label"]
            ratings.append(rating.replace("Rated ", "").replace(" stars", ""))
        else:
            ratings.append("None")

        #product_urls
        if sales.select_one("a.brand-and-name") is not None:
            product_url = sales.select_one("a.brand-and-name")
            product_url = product_url["href"]
            product_urls.append("https://www.macys.com/" + product_url)

        else:
            product_urls.append("None")

     #We can also use the find as well       
    #     if sales.find("a", class_ = "brand-and-name") is not None:
    #         product_url = sales.find("a", class_ = "brand-and-name")
    #         product_url = product_url["href"]
    #         product_urls.append("https://www.macys.com/" + product_url)
  
    if soup.find("a", class_ = "next-button") is None:
        return False
    else:
        return True
    
base_url, query_string = url.split("?")
index = 1
    
while True:
    try:
        pages = "/Pageindex/" + "{}?{}".format(index, query_string)
        index += 1
        new_url = base_url + pages
        should_continue = scrape_macy(new_url)
        print(new_url)
        print(len(product_urls))
            
    except Exception as e:
        print(e)
        break

sales_df = pd.DataFrame(
{"product_brand": product_brands,
 "product_name": product_names,
 "current_price": current_prices,
 "old_price": old_prices,
 "rating": ratings,
 "product_url": product_urls

}
)


sales_df

https://www.macys.com/shop/womens-clothing/accessories/Pageindex/1?id=29440
120
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/2?id=29440
240
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/3?id=29440
360
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/4?id=29440
480
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/5?id=29440
600
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/6?id=29440
720
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/7?id=29440
840
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/8?id=29440
960
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/9?id=29440
1080
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/10?id=29440
1200
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/11?id=29440
1258
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/12?id=29440
1258
https://www.macys.com/shop/womens

KeyboardInterrupt: 

In [93]:
sales_df = sales_df.drop_duplicates(subset = ["product_url"])
sales_df = sales_df.reset_index(drop=True)

In [94]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_brand  60 non-null     object
 1   product_name   60 non-null     object
 2   current_price  60 non-null     object
 3   old_price      60 non-null     object
 4   rating         60 non-null     object
 5   product_url    60 non-null     object
dtypes: object(6)
memory usage: 2.9+ KB


In [ ]:
sales_df